In [ ]:
import numpy as np
import matplotlib.pyplot as pl
from functools import partial

import scipy.integrate as integrate

#### Gammatone

$s(t)= t_+^{k-1} e^{-t/\tau}$

$||s||_2^2= (\tau/2)^{2k-1} (2k-2)!$

In [ ]:
def gammatone(k, tau, t):
    s2_2 = np.power(tau/2, 2*k-1)*np.math.factorial(2*k-2)
    return np.sqrt(1/s2_2)*(t>0)*np.power(t, k-1)*np.exp(-t/tau)

In [ ]:
k=4
tau=1
gammatone_4=partial(gammatone, k, tau)

In [ ]:
t=np.linspace(-0.2, 10)
pl.plot(t, gammatone_4(t))

In [ ]:
#test normalization
#integrate.quad(lambda x : gammatone_4(x)**2, 0, 40)

#### Power spectrum

$\hat{s}(\omega)=(k-1)! \left[ \frac{\tau}{1-i\omega\tau} \right]^{k}$

$|\hat{s}(\omega)|^2= (k-1)!^2 \left[ \frac{\tau^2}{1+\omega^2\tau^2} \right]^{k}$

In [ ]:
def gammatone_freq(k, tau, omega):
    s2_2 = np.power(tau/2, 2*k-1)*np.math.factorial(2*k-2)
    return np.sqrt(1/s2_2)*np.math.factorial(k-1)*np.power(tau/(1-1j*tau*omega), k)

def gammatone_freq_abs(k, tau, omega):
    s2_2 = np.power(tau/2, 2*k-1)*np.math.factorial(2*k-2)
    return np.sqrt(1/s2_2)*np.math.factorial(k-1)*np.power(tau**2/(1+tau**2*omega**2), k/2)


def gammatone_freq_abs_sq(k, tau, omega):
    s2_2 = np.power(tau/2, 2*k-1)*np.math.factorial(2*k-2)
    return 1/s2_2*np.math.factorial(k-1)**2*np.power(tau**2/(1+tau**2*omega**2), k)



In [ ]:
t=np.linspace(0, 30, 1024)
u=gammatone_4(t)
v=np.fft.rfft(u)

dt=t[1]-t[0]

f=1/t[-1]*np.arange(len(t)//2+1)
#v_comp=gammatone_freq(k, tau, 2*np.pi*f)
#v_comp_abs=gammatone_freq_abs(k, tau, 2*np.pi*f)
v_comp_abs2=np.sqrt(gammatone_freq_abs_sq(k, tau, 2*np.pi*f))

pl.plot(f, dt*np.abs(v), linewidth=3)

#pl.plot(f, np.abs(v_comp), '-.', linewidth=3)
pl.plot(f, v_comp_abs, '-.', linewidth=3)
pl.xlim([0,1.5])

#### Integration

from matlab, primitives of $cos^{2k}$

```
syms x
f=cos(x)^2
int(f)
```
 * $k=1$: $x/2 + \sin(2x)/4$
 * $k=2$: $(3x)/8 + \sin(2x)/4 + \sin(4x)/32$
 * $k=3$: $(5x)/16 + (15\sin(2x))/64 + (3\sin(4x))/64 + \sin(6x)/192$
 * $k=4$: $(35x)/128 + (7\sin(2x))/32 + (7\sin(4x))/128 + \sin(6x)/96 + \sin(8x)/1024$

In [ ]:
def prim(k, x):
    if k==1:
        return x/2 + np.sin(2*x)/4
    elif k==2:
        return (3*x)/8 + np.sin(2*x)/4 + np.sin(4*x)/32
    elif k==3:
        return (5*x)/16 + (15*np.sin(2*x))/64 + (3*np.sin(4*x))/64 + np.sin(6*x)/192

$\int_0^A (1+\omega^2 \tau^2)^{-k} d\omega = \frac{1}{\tau} \int_0^{\mathrm{arctan}(\tau A)} \cos^{2(k-1)}\theta   \, d\theta$